In [ ]:
#pip install neptune-sklearn

In [3]:
#pip install lazypredict --user

  Using cached lazypredict-0.2.9-py2.py3-none-any.whl (12 kB)
  Using cached pytest-5.4.3-py3-none-any.whl (248 kB)
  Using cached lightgbm-2.3.1-py2.py3-none-win_amd64.whl (544 kB)
  Using cached PyYAML-5.3.1-cp38-cp38-win_amd64.whl (219 kB)
  Using cached scikit_learn-0.23.1-cp38-cp38-win_amd64.whl (6.8 MB)
  Using cached pandas-1.0.5-cp38-cp38-win_amd64.whl (8.9 MB)
  Using cached joblib-1.0.0-py3-none-any.whl (302 kB)
  Using cached xgboost-1.1.1-py3-none-win_amd64.whl (54.4 MB)
  Using cached pluggy-0.13.1-py2.py3-none-any.whl (18 kB)
  Using cached atomicwrites-1.4.0-py2.py3-none-any.whl (6.8 kB)
  Using cached more_itertools-8.7.0-py3-none-any.whl (48 kB)
  Using cached py-1.10.0-py2.py3-none-any.whl (97 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
neptune-sklearn 0.9.1 requires scikit-learn>=0.24.1, but you have scikit-learn 0.23.1 which is incompatible.


In [51]:
# библиотеки для lazypredict
import pandas as pd
from lazypredict.Supervised import LazyRegressor
from pandas.plotting import scatter_matrix
# Scikit-learn packages
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
# Hide warnings
import warnings
warnings.filterwarnings("ignore")
# Setting up max columns displayed to 100
pd.options.display.max_columns = 100

In [52]:
#импортируем библиотеки
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import statsmodels.formula.api as sf
import numpy as np
from sklearn.metrics import mean_squared_error #оценка MSE полученных моделей
from sklearn.metrics import r2_score #оценка MSE полученных моделей
from sklearn.model_selection import train_test_split  # метод деления на train/test выборки

In [53]:
from tqdm.auto import tqdm
from statsmodels.graphics.gofplots import qqplot #оценка нормальности распределения через QQ-plot
from scipy.stats import shapiro #оценка нормальности распределения через тетс Шапиро-Уилка


In [54]:
#подключение к системе логирования экспериментов neptune.ai
import neptune.new as neptune



In [55]:
#считываем данные
df = pd.read_excel(r'g:\Yandex\Задание_вып.xls', sheet_name='Мини моделирование', header=1)

#немного переименовываем колонки для удобства работы
df.rename(columns={'Общие конверсии':'Общие_конверсии',
                     'Автобусы/Кинотеатры/Лифты/Метро OTS':'Автобусы',
                     'ООН/Радио/Транспорт/DООН/Digital/Indoor OTS':'ООН',
                  'Интерес к поиску работы':'поиск_работы',
                  'Интерес к бренду':'бренд',
                  'Интерес к брендам конкурентов':'конкуренты',
                  'Индекс самоизоляции':'самоизоляции',
                  'Безработица_Все гео':'Безработица',
                  'Гос праздники + НГ':'праздники',
                  'Созданные компании':'Созданные',
                  'Ликвидированные компании':'Ликвидированные',
                  'Запросы Гугл Трендс':'Гугл_Трендс'}, inplace=True)

y = df['Общие_конверсии']

X0=df.drop(['Общие_конверсии'], axis = 1)

X=df.drop(['Общие_конверсии', 'Дата'], axis = 1)


In [56]:
# сплиттим данные train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = 23)

In [57]:
# используем LazyRegressor для проверки лучших моделей
reg = LazyRegressor(ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
print(models)

100%|██████████████████████████████████████████| 42/42 [00:05<00:00,  8.15it/s]

                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
ExtraTreesRegressor                          0.96       0.97  19129.63   
GradientBoostingRegressor                    0.95       0.96  20640.48   
XGBRegressor                                 0.95       0.95  21802.98   
LGBMRegressor                                0.94       0.95  22488.77   
HistGradientBoostingRegressor                0.94       0.95  22549.34   
RandomForestRegressor                        0.94       0.95  22635.88   
BaggingRegressor                             0.93       0.94  25428.54   
ExtraTreeRegressor                           0.92       0.93  27371.07   
DecisionTreeRegressor                        0.91       0.92  28669.83   
AdaBoostRegressor                            0.90       0.91  30657.43   
LinearRegression                             0.88       0.90  32852.98   
TransformedTargetRegressor            

In [58]:
# удаляем одну из колонок с корреляцией = 1 
X=X.drop(['конкуренты'], axis = 1)

In [59]:
# сплиттим измененные данные train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = 23)

In [60]:
# еще раз используем LazyRegressor для проверки лучших моделей на измененных данных
reg = LazyRegressor(ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
print(models)

100%|██████████████████████████████████████████| 42/42 [00:05<00:00,  8.39it/s]

                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
ExtraTreesRegressor                          0.96       0.97  18075.75   
GradientBoostingRegressor                    0.95       0.96  20674.17   
XGBRegressor                                 0.95       0.95  21802.98   
LGBMRegressor                                0.94       0.95  22488.77   
RandomForestRegressor                        0.94       0.95  22493.40   
HistGradientBoostingRegressor                0.94       0.95  22549.34   
BaggingRegressor                             0.94       0.95  23502.00   
DecisionTreeRegressor                        0.89       0.91  31119.87   
AdaBoostRegressor                            0.89       0.90  31619.42   
ExtraTreeRegressor                           0.88       0.90  32675.74   
TransformedTargetRegressor                   0.88       0.90  32852.98   
LinearRegression                      

In [61]:
# Evaluation Functions
def rmse(model, y_test, y_pred, X_test):
    r_squared = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print('R-squared: ' + str(r_squared))
    print('Mean Squared Error: '+ str(rmse))
    return r_squared, rmse
# Create model line scatter plot
def scatter_plot(y_test, y_pred, model_name):
    plt.figure(figsize=(10,6))
    sns.residplot(y_test, y_pred, lowess=True, color='#4682b4',
              line_kws={'lw': 2, 'color': 'r'})
    plt.title(str('Общие конверсии vs Residuals for '+ model_name))
    plt.xlabel('Price',fontsize=16)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.show()

Для оценки R2 и RMSE используем выборку в 1000 итераций с различным разделением на train/test. 
Random_state для повторяемости примем во всех случаях i*3.
Данный подход позволит определить границы доверительного интервала для вероятности 95%.

In [62]:
# перепроверим первый регрессор
from sklearn.ensemble import ExtraTreesRegressor

In [73]:
r2_array=[]
rmse_array = []

for i in tqdm(range(1000)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = i*3)
    ETRreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
    ETRreg.fit(X_train, y_train)
    y_pred = ETRreg.predict(X_test)
    r2_array.append(ETRreg.score(X_test, y_test))
    rmse_array.append(np.sqrt(mean_squared_error(y_test, y_pred)))
r2_ETR = np.mean(r2_array)
rmse_ETR = np.mean(rmse_array)
print('Average ETR R-squared: ' + str(r2_ETR))
print('Average ETR R-squared_min (confidence=0.95): ' + str(np.quantile(r2_array, 0.05)))
print('Average ETR R-squared_max (confidence=0.95): ' + str(np.quantile(r2_array, 0.95)))
print('Average ETR Root Mean Squared Error: '+ str(rmse_ETR))
print('Average ETR Root Mean Squared Error_min (confidence=0.95): '+ str(np.quantile(rmse_array, 0.05)))
print('Average ETR Root Mean Squared Error_max (confidence=0.95): '+ str(np.quantile(rmse_array, 0.95)))

  0%|          | 0/1000 [00:00<?, ?it/s]

Average ETR R-squared: 0.9707100551453166
Average ETR R-squared_min (confidence=0.95): 0.9605307522648614
Average ETR R-squared_max (confidence=0.95): 0.9793248182624625
Average ETR Root Mean Squared Error: 18012.040868902466
Average ETR Root Mean Squared Error_min (confidence=0.95): 15292.672461793449
Average ETR Root Mean Squared Error_max (confidence=0.95): 20910.416020625325


In [67]:
# перепроверим второй регрессор
from sklearn.ensemble import GradientBoostingRegressor

In [68]:
r2_array=[]
rmse_array = []

for i in tqdm(range(1000)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = i*3)
    GBRreg = GradientBoostingRegressor(n_estimators=100, random_state=0)
    GBRreg.fit(X_train, y_train)
    y_pred = GBRreg.predict(X_test)
    r2_array.append(GBRreg.score(X_test, y_test))
    rmse_array.append(np.sqrt(mean_squared_error(y_test, y_pred)))
r2_GBR = np.mean(r2_array)
rmse_GBR = np.mean(rmse_array)
print('Average GBR R-squared: ' + str(r2_GBR))
print('Average GBR R-squared_min (confidence=0.95): ' + str(np.quantile(r2_array, 0.05)))
print('Average GBR R-squared_max (confidence=0.95): ' + str(np.quantile(r2_array, 0.95)))
print('Average GBR Root Mean Squared Error: '+ str(rmse_GBR))
print('Average GBR Root Mean Squared Error_min (confidence=0.95): '+ str(np.quantile(rmse_array, 0.05)))
print('Average GBR Root Mean Squared Error_max (confidence=0.95): '+ str(np.quantile(rmse_array, 0.95)))


  0%|          | 0/1000 [00:00<?, ?it/s]

Average GBR R-squared: 0.9649154679897104
Average GBR R-squared_min (confidence=0.95): 0.9520851239885215
Average GBR R-squared_max (confidence=0.95): 0.9750797553321364
Average GBR Mean Squared Error: 19717.410120077468
Average GBR Mean Squared Error_min (confidence=0.95): 16878.190971228447
Average GBR Mean Squared Error_max (confidence=0.95): 22865.392723381883


In [69]:
# перепроверим третий регрессор
import xgboost as xgb 

In [70]:
r2_array=[]
rmse_array = []

for i in tqdm(range(1000)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = i*3)
    XGBRreg = xgb.XGBRegressor(verbosity=0)
    XGBRreg.fit(X_train, y_train)
    y_pred = XGBRreg.predict(X_test)
    r2_array.append(XGBRreg.score(X_test, y_test))
    rmse_array.append(np.sqrt(mean_squared_error(y_test, y_pred)))
r2_XGBR = np.mean(r2_array)
rmse_XGBR = np.mean(rmse_array)
print('Average XGBR R-squared: ' + str(r2_XGBR))
print('Average XGBR R-squared_min (confidence=0.95): ' + str(np.quantile(r2_array, 0.05)))
print('Average XGBR R-squared_max (confidence=0.95): ' + str(np.quantile(r2_array, 0.95)))
print('Average XGBR Root Mean Squared Error: '+ str(rmse_XGBR))
print('Average XGBR Root Mean Squared Error_min (confidence=0.95): '+ str(np.quantile(rmse_array, 0.05)))
print('Average XGBR Root Mean Squared Error_max (confidence=0.95): '+ str(np.quantile(rmse_array, 0.95)))

  0%|          | 0/1000 [00:00<?, ?it/s]

Average XGBR R-squared: 0.9653099092239329
Average XGBR R-squared_min (confidence=0.95): 0.9516747274661159
Average XGBR R-squared_max (confidence=0.95): 0.9756920125052039
Average XGBR Mean Squared Error: 19590.029434307264
Average XGBR Mean Squared Error_min (confidence=0.95): 16657.97652759966
Average XGBR Mean Squared Error_max (confidence=0.95): 22930.122564464


Лучший средний R2 и RMSE показал ExtraTreesRegressor.

In [ ]:
# подбор параметров для ExtraTreesRegressor

In [74]:
from sklearn.model_selection import GridSearchCV

In [75]:
# сплиттим измененные данные train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = 23)

In [76]:
clf = ExtraTreesRegressor()

In [77]:
parametrs = { 'n_estimators': range (10, 101, 10),
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

In [80]:
grid = GridSearchCV(clf, parametrs, cv=5, verbose = 2)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20 
[CV]  max_depth=1, min_s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=30, total=   0.

[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=20, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=20 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=20, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=4, n_estimators=30

[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=20, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=20 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=20, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30 
[CV]  max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30, total=   0.0s
[CV] max_depth=1, min_samples_leaf=1, min_samples_split=6, n_estimators=30

...

[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=30 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=30 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=4, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=4, n_e

[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=30 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=30 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=6, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=6, n_e

[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=30 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=30 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=30, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=40 
[CV]  max_depth=11, min_samples_leaf=7, min_samples_split=8, n_estimators=40, total=   0.1s
[CV] max_depth=11, min_samples_leaf=7, min_samples_split=8, n_e

[Parallel(n_jobs=1)]: Done 8400 out of 8400 | elapsed: 15.2min finished


GridSearchCV(cv=5, estimator=ExtraTreesRegressor(),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(10, 101, 10)},
             verbose=2)

In [82]:
best_params = grid.best_params_

In [84]:
best_params

{'max_depth': 11,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 30}

In [90]:
r2_array=[]
rmse_array = []

for i in tqdm(range(1000)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = i*3)
    ETRreg = ExtraTreesRegressor(random_state=0)
    ETRreg.set_params(**best_params)
    ETRreg.fit(X_train, y_train)
    y_pred = ETRreg.predict(X_test)
    r2_array.append(ETRreg.score(X_test, y_test))
    rmse_array.append(np.sqrt(mean_squared_error(y_test, y_pred)))
r2_ETR = np.mean(r2_array)
rmse_ETR = np.mean(rmse_array)
print('Average ETR R-squared: ' + str(r2_ETR))
print('Average ETR R-squared_min (confidence=0.95): ' + str(np.quantile(r2_array, 0.05)))
print('Average ETR R-squared_max (confidence=0.95): ' + str(np.quantile(r2_array, 0.95)))
print('Average ETR Root Mean Squared Error: '+ str(rmse_ETR))
print('Average ETR Root Mean Squared Error_min (confidence=0.95): '+ str(np.quantile(rmse_array, 0.05)))
print('Average ETR Root Mean Squared Error_max (confidence=0.95): '+ str(np.quantile(rmse_array, 0.95)))

  0%|          | 0/1000 [00:00<?, ?it/s]

Average ETR R-squared: 0.9680755568142063
Average ETR R-squared_min (confidence=0.95): 0.9561529224300219
Average ETR R-squared_max (confidence=0.95): 0.9774558843318129
Average ETR Root Mean Squared Error: 18801.719834594507
Average ETR Root Mean Squared Error_min (confidence=0.95): 15988.248040099877
Average ETR Root Mean Squared Error_max (confidence=0.95): 21927.384318233726


In [91]:
ETRreg.feature_importances_

array([0.00221804, 0.00794835, 0.00396481, 0.00249713, 0.00140369,
       0.00249127, 0.0051936 , 0.00325849, 0.00358529, 0.00509941,
       0.00695246, 0.23777444, 0.53230367, 0.07190417, 0.01505273,
       0.00212848, 0.03448573, 0.04671548, 0.01502276])